# Coursera Capstone

### This notebook will be mainly used for the capstone project for IBM Data Science Specialization
#### Assignment Week 1: Capstone Project Notebook

In [130]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


#### Assignment Week 3: Segmenting and Clustering Neighborhoods in Toronto

*a) Downloading the dataset*

In [131]:
from pandas.io.html import read_html
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(wiki_url,  attrs={"class":"wikitable"})
wikitable[0]

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


*b) Ignoring cells with a borough that is Not assigned*

In [132]:
df = wikitable[0][wikitable[0].Borough != 'Not assigned']
df

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


*c) Combining several rows into one row with the neighborhoods for one postal cade*

In [133]:
df=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(list)
df=df.sample(frac=1).reset_index()
df['Neighborhood']=df['Neighborhood'].str.join(',')
df.head()

,Postcode,Borough,Neighborhood
0,M9L,North York,Humber Summit
1,M5M,North York,"Bedford Park,Lawrence Manor East"
2,M4T,Central Toronto,"Moore Park,Summerhill East"
3,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel"
4,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market"


*d) Assigning the neighborhood as borough for not assigned neighborhood*

In [134]:
df.loc[df['Neighborhood'] == "Not assigned"] = df['Borough']

*e) Printing the shape of the dataframe*

In [135]:
df.shape

(103, 3)

 *Creating the Dataframe to get the coordinates for all of neighborhoods*

In [169]:
latlong=pd.read_csv('Geospatial_Coordinates.csv')
latlong=latlong.rename(columns={'Postal Code':'Postcode'})
df_geo=pd.merge(df, latlong, on='Postcode')
df_geo

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M9L,North York,Humber Summit,43.756303,-79.565963
1,M5M,North York,"Bedford Park,Lawrence Manor East",43.733283,-79.419750
2,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
3,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817
4,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049
...,...,...,...,...,...
97,M3B,North York,Don Mills North,43.745906,-79.352188
98,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
99,M4J,East York,East Toronto,43.685347,-79.338106
100,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
